In [5]:
# widen jupyter notebook window
from IPython.core.display import display, HTML
display(HTML("<style>.container {width:95% !important; }</style>"))

In [45]:
import torch
from torch.utils.data import DataLoader

import numpy as np
import tensorly as tl
import scipy.signal
import matplotlib.pyplot as plt
import copy

In [7]:
import sys
sys.path.append(r'/media/rich/Home_Linux_partition/github_repos')
%load_ext autoreload
%autoreload 2
from tensor_regression import standard_tensor_regression as STR

In [8]:
import tensorly as tl

In [9]:
%load_ext autoreload
%autoreload 2
import basic_neural_processing_modules as bnpm
from basic_neural_processing_modules import torch_helpers
from basic_neural_processing_modules import h5_handling
from basic_neural_processing_modules import misc

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
dir_data = r'/media/rich/bigSSD/for_Josh/'
fileName_X_data = r'positions_convDR_meanSub_s2pInd.h5'
fileName_y_data = r'neural_data.h5'
positions = h5_handling.simple_load(directory=dir_data, fileName=fileName_X_data)
neural_data = h5_handling.simple_load(directory=dir_data, fileName=fileName_y_data)

In [43]:
torch_helpers.show_cuda_devices()
DEVICE = STR.set_device(use_GPU=True, verbose=True)

1 device(s) found.
0 GeForce RTX 3090
GPU is enabled.


In [47]:
from torch.utils.data import Dataset

class WindowedDataset(Dataset):
    def __init__(self, X_untiled, y_input, win_range, transform=None, target_transform=None):
        self.X_untiled = X_untiled # first dim will be subsampled from
        self.y_input = y_input # first dim will be subsampled from
        self.win_range = win_range
        self.n_samples = y_input.shape[0]
        self.usable_idx = torch.arange(-self.win_range[0] , self.n_samples-self.win_range[1]+1)
        
        if X_untiled.shape[0] != y_input.shape[0]:
            raise ValueError('RH: X and y must have same first dimension shape')

    def __len__(self):
        return self.n_samples
    
    def check_bound_errors(self, idx):
        idx_toRemove = []
        for val in idx:
            if (val+self.win_range[0] < 0) or (val+self.win_range[1] > self.n_samples):
                idx_toRemove.append(val)
        if len(idx_toRemove) > 0:
            raise ValueError(f'RH: input idx is too close to edges. Remove idx: {idx_toRemove}')

    def __getitem__(self, idx):
#         print(idx)
#         self.check_bound_errors(idx)
        X_subset_tiled = self.X_untiled[idx+self.win_range[0] : idx+self.win_range[1]]
        y_subset = self.y_input[idx]
        return X_subset_tiled, y_subset

In [50]:
pos_array = torch.tensor(positions['positions_convDR_meanSub_s2pInd'])
print(f'pos_array.shape = {pos_array.shape}')

ic = neural_data['is_cell']
good_ROIs = neural_data['good_ROIs']
neural_array = torch.tensor(neural_data['dFoF'][ic][good_ROIs]).T

print(f'neural_array.shape = {neural_array.shape}')

win_range = [-120*3, -120*-1]

pos_array.shape = torch.Size([108000, 2744])
neural_array.shape = torch.Size([108000, 811])


In [61]:
def make_WindowedDataloader(X, y, win_range):
    dataset = WindowedDataset(X, y, win_range)

    sampler = torch.utils.data.SubsetRandomSampler(dataset.usable_idx, generator=None)

    dataloader = DataLoader(dataset, batch_size=3000,
        shuffle=False, pin_memory=False, drop_last=True, num_workers=0,
                            sampler=sampler)
    return dataloader, dataset, sampler

In [62]:
dataloader, dataset, sampler = make_WindowedDataloader(pos_array, neural_array, win_range)

In [63]:
import time

# test.auto_collation=False
tic=time.time()
for val in dataloader:
#     print('hii')
# for val in dataset:
#     val
    val[0].to(DEVICE)
    val[1].to(DEVICE)
    print(val[0].shape)
    print(misc.estimate_size_of_float_array(input_shape=val[0].shape, bitsize=32)/1000000000)
#     print(val[0].dtype)
#     print(val[0].device)
    print(time.time() - tic)
    tic = time.time()
#     print(len(val[0]))
#     print(val[0].shape)

torch.Size([3000, 480, 2744])
15.80544
3.8548481464385986


KeyboardInterrupt: 